In [4]:
from langchain_unstructured import UnstructuredLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
import bs4
from bs4 import BeautifulSoup
import requests
import urllib.request
from tqdm import tqdm
from langchain_google_genai import (
    HarmBlockThreshold,
    HarmCategory,
)
import os
from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import BeautifulSoupTransformer

In [91]:
urls = []

prefix = "https://docs.manim.community/en/stable/"

with open("links.txt", "r") as f:
    for line in f:
        urls.append(prefix + line.strip())

print(urls)

['https://docs.manim.community/en/stable/reference_index/animations.html', 'https://docs.manim.community/en/stable/reference/manim.animation.animation.html', 'https://docs.manim.community/en/stable/reference/manim.animation.animation.Animation.html', 'https://docs.manim.community/en/stable/reference/manim.animation.animation.Wait.html', 'https://docs.manim.community/en/stable/reference/manim.animation.changing.html', 'https://docs.manim.community/en/stable/reference/manim.animation.changing.AnimatedBoundary.html', 'https://docs.manim.community/en/stable/reference/manim.animation.changing.TracedPath.html', 'https://docs.manim.community/en/stable/reference/manim.animation.composition.html', 'https://docs.manim.community/en/stable/reference/manim.animation.composition.AnimationGroup.html', 'https://docs.manim.community/en/stable/reference/manim.animation.composition.LaggedStart.html', 'https://docs.manim.community/en/stable/reference/manim.animation.composition.LaggedStartMap.html', 'http

In [ ]:
for url in tqdm(urls):
    try:
        r = requests.get(url)

        if r.status_code != 200:
            print(f"Failed to fetch {url}")
            continue

        soup = BeautifulSoup(r.text, "html.parser")
        print(f"Title: {soup.title.text}")
    

    except Exception as e:
        print("Failed.")
        print(f"Details: {e}")

In [93]:
def get_text(url):
    fp = urllib.request.urlopen(url)
    mybytes = fp.read()

    mystr = mybytes.decode("utf8")
    fp.close()

    soup = BeautifulSoup(mystr)


    for code in soup.find_all(class_="pre"):
        code.string.replace_with("\`\`\`"+code.string+"\`\`\`")
        if "[source]" in code.string:
            code.string.replace_with("")

    for code in soup.find_all():
        if code.name == "pre" and code.string:
            code.decompose()

    for code in soup.find_all(class_="highlight"):
        for pre in code.find_all("pre"):

            total = len(pre.find_all("span"))
            caught = False

            for span in pre.find_all("span"):
                if not span.string:
                    continue

                if not caught:
                    span.string.replace_with("\`\`\`"+span.string)
                    caught = True

                last = span
            
            last.string.replace_with(last.string + "\`\`\`")



    return soup.article.get_text()


In [96]:
i = 0
for url in tqdm(urls):
    try:
        text = get_text(url)
        with open(f"localManim/{i}.txt", "w", encoding="utf-8") as file:
            file.write(text)
        i += 1
    except Exception as e:
        print("Failed.")
        print(f"Details: {e}")

100%|██████████| 174/174 [00:48<00:00,  3.56it/s]
